In [1]:
from shared.preface import *
import shared.functions as fct

# Initialize parameters and files.
PRE = PRE(
    # sim='L006N188', 
    sim='L012N376', 
    z0_snap=62, z4_snap=13, DM_lim=1000,
    sim_dir=SIM_ROOT, sim_ver=SIM_TYPE,
    phis=10, thetas=10, vels=400,
    pre_CPUs=6, sim_CPUs=6
)

mass_gauge = 12.3
mass_range = 0.3
size = 1
hname = f'1e+{mass_gauge}_pm{mass_range}Msun'
fct.halo_batch_indices(
    PRE.Z0_STR, mass_gauge, mass_range, 'halos', size, 
    hname, PRE.SIM_DIR, PRE.OUT_DIR
)
halo_batch_IDs = np.load(f'{PRE.OUT_DIR}/halo_batch_{hname}_indices.npy')
halo_batch_params = np.load(f'{PRE.OUT_DIR}/halo_batch_{hname}_params.npy')
halo_num = len(halo_batch_params)

print('********Number density band********')
print('Halo batch params (Rvir,Mvir,cNFW):')
print(halo_batch_params)
print('***********************************')


mass_ticks = 2
mass_range_eV = np.geomspace(0.01, 0.3, mass_ticks)*eV  # 10 to 300 meV

n_nus_l = []
for halo_j in range(halo_num):
# for halo_j in (0,2):
    
# halo_j = 0
# halo_num = 1

    # Load velocities.
    tot_name = f'{hname}_halo{halo_j}'
    fname = f'{PRE.NUS}nus_{tot_name}'
    u_all = fct.load_sim_data(PRE.OUT_DIR, fname, 'velocities')

    # Calculate number density.
    out_file = f'{PRE.OUT_DIR}/number_densities_{PRE.NUS}nus_{tot_name}.npy'
    
    # "Normal" method.
    fct.number_density_1_mass(
        u_all, mass_range_eV, out_file, 
        average=False, m_average=0.01, z_average=3.5
    )

    # Using medians or min of momenta batches.
    # fct.number_density_1_mass_median(
    #     u_all, mass_range_eV, out_file,
    #     PRE.PHIs, PRE.THETAs, PRE.Vs
    # )

    # No clustering number density, but using discrete integration,
    # instead of analytical formula.
    norm_file = f'{PRE.OUT_DIR}/norms_{PRE.NUS}nus_{tot_name}.npy'
    fct.number_density_discrete(u_all, mass_range_eV, norm_file, 100)
    norm = np.load(f'{norm_file}')

    # nrd = np.load(f'{out_file}')/N0
    nrd = np.load(f'{out_file}')/norm

    n_nus_l.append(nrd)


n_nus_arr = np.array(n_nus_l).reshape(halo_num, mass_ticks)


# Load data for MW halo only of smooth simulation
nus_smooth = 10000
MW_HALO = True
VC_HALO = False
AG_HALO = False
halos = 'MW'*MW_HALO + '+VC'*VC_HALO + '+AG'*AG_HALO
sim = 'LinfNinf'
smooth_name = f'nus_{nus_smooth}_halos_{halos}_{SOLVER}'
u_all_MW = fct.load_sim_data(sim, smooth_name, 'velocities')

output = f'{sim}/number_densities_{smooth_name}.npy'
fct.number_density_1_mass(
    u_all_MW, mass_range_eV, output, 
    average=False, m_average=0.1, z_average=2.0
    )
n_nus_MW = np.load(f'{output}')/N0

********************* Initialization *********************
# Initial conditions for neutrinos:
PHIs = 10, THETAs=10, Vs=400
Total neutrinos: 40000
# Simulation parameters:
Simulation box: L012N376
Snapshot from 0062 (z=0) to 0013 (z=4)
Pre/Sim CPUs 6/6
DM limit for cells: 1000
# File management:
Box files directory: 
 /home/fabian/ownCloud/snellius/L012N376/CDM_TF50
Output directory: 
 /home/fabian/my_github_projects/neutrino_clustering_V2/L012N376/CDM_TF50
**********************************************************
********Number density band********
Halo batch params (Rvir,Mvir,cNFW):
[[299.54209457  12.45775108   8.69090077]]
***********************************
[5.58791954e-07 5.58791954e-07 5.58791954e-07 ... 2.18134225e-03
 2.18134225e-03 2.18134225e-03]
[1.67637586e-05 1.67637586e-05 1.67637586e-05 ... 6.54402674e-02
 6.54402674e-02 6.54402674e-02]
[5.58791955e-08 5.58791955e-08 5.58791955e-08 ... 2.18134225e-03
 2.18134225e-03 2.18134225e-03]
[1.67637586e-06 1.67637586e-06 1.6763

In [ ]:
fig, ax = plt.subplots(1,1)
fig.patch.set_facecolor('cornflowerblue')
# ax.patch.set_facecolor('lightskyblue')

# Plot smooth simulation.
ax.plot(
    mass_range_eV*1e3, (n_nus_MW-1), color='red', ls='solid', 
    label='Analytical simulation'
)

nus_median = np.median(n_nus_arr, axis=0)
nus_perc2p5 = np.percentile(n_nus_arr, q=2.5, axis=0)
nus_perc97p5 = np.percentile(n_nus_arr, q=97.5, axis=0)
nus_perc16 = np.percentile(n_nus_arr, q=16, axis=0)
nus_perc84 = np.percentile(n_nus_arr, q=84, axis=0)

# Plotting the number density median and percentiles, to create a "band".
ax.plot(
    mass_range_eV*1e3, (nus_median-1), color='blue', 
    label='N-body simulation (medians of halo batch)'
)
ax.fill_between(
    mass_range_eV*1e3, (nus_perc2p5-1), (nus_perc97p5-1), 
    color='blue', alpha=0.2, label='2.5-97.5 %')
ax.fill_between(
    mass_range_eV*1e3, (nus_perc16-1), (nus_perc84-1), 
    color='blue', alpha=0.3, label='16-84 %')


# Plot approximate values (a line) from Mertsch et al.
# x_ends = [1e1, 3*1e2]
# y_ends = [3*1e-3, 4]
# ax.plot(x_ends, y_ends, marker='x', ls=':', c='r', alpha=0.6)

# for m in NU_MASSES:
#     ax.axvline(m*1e3, c='r', ls='-.')

ax.set_xscale('log')
ax.set_yscale('log')
# ax.set_title(f'{nus}nus ; {halo_num} halos of {batchname}')
ax.set_title(f'Number densities with MW-type halo sample')  # nice title
ax.set_xlabel(r'$m_{\nu}$ [meV]')
ax.set_ylabel(r'$n_{\nu} / n_{\nu, 0}$')
# ax.set_ylim(1e-3, 1e1)
plt.grid(True, which="both", ls="-")
plt.legend(loc='lower right')

ax.yaxis.set_major_formatter(ticker.FuncFormatter(fct.y_fmt))

plt.savefig(
    f'figures/{SIM_TYPE}_eta_band_{PRE.NUS}nus_{hname}_batch.pdf',
    facecolor=fig.get_facecolor(), edgecolor='none', bbox_inches='tight'
    )
plt.show()

# print('Max value:', np.max(n_nus), np.max(n_nus-1))

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(12,12))
fig.suptitle(
    'Momentum phase-space distributions' 
    '\n'
    f'({PRE.NUS} neutrinos)',
    fontsize=18)

Test_masses = np.array([0.02,0.021,0.022,0.023])*eV
# u_all = u_all_MW

for j, m_eV in enumerate(NU_MASSES):

    k = j

    # Convert to momenta.
    p, y = fct.u_to_p_eV(u_all, m_eV)
    p0, p1, y0, y1 = p[:,0], p[:,-1], y[:,0], y[:,-1]

    # Sort.
    order = p0.argsort()
    p0_sort, p1_sort = p0[order], p1[order]
    y0_sort, y1_sort = y0[order], y1[order]

    curtain_behaviour = False

    if curtain_behaviour:
        p1_final = p1_sort
        y0_final = y0_sort
    else:
        # Each velocity has a batch of neutrinos!
        # Take min. of each to represent most clustered ones.
        p1_blocks = p1_sort.reshape((PRE.Vs, PRE.PHIs*PRE.THETAs))
        p1_final = np.min(p1_blocks, axis=1)
        y0_blocks = y0_sort.reshape((PRE.Vs, PRE.PHIs*PRE.THETAs))
        y0_final = y0_blocks[:,0]

    # Fermi Dirac of the smoothed final momenta.
    # FDvals = fct.Fermi_Dirac(p1_smooth)
    FDvals = fct.Fermi_Dirac(p1_final)

    i = 0
    if j in (2,3):
        i = 1
        j -= 2
    
    # Escape momentum.
    Rvir_halo = halo_batch_params[0,0]*kpc
    Mvir_halo = 10**(halo_batch_params[0,1])*Msun
    cNFW_halo = halo_batch_params[0,2]
    rho0_halo = fct.scale_density_NFW(0., cNFW_halo)*(Msun/kpc**3)
    rs_halo = Rvir_halo/cNFW_halo
    p_esc_eV, y_esc = fct.escape_momentum(
        X_SUN, 0., rho0_halo, Mvir_halo, Rvir_halo, rs_halo, m_eV
    )
    axs[i,j].axvline(y_esc, c='k', ls='-.', label='y_esc')

    # Original Fermi-Dirac distr.
    pOG = np.geomspace(PRE.LOWER, PRE.UPPER, len(FDvals))
    FDvalsOG = fct.Fermi_Dirac(pOG)
    yOG = pOG/T_CNB

    axs[i,j].loglog(y0_final, FDvals, label='Simulation', c='red', alpha=0.9)
    axs[i,j].loglog(yOG, FDvalsOG, label='Fermi-Dirac', c='blue', alpha=0.7)
    axs[i,j].set_title(f'{m_eV} eV')
    axs[i,j].set_ylabel('FD(p)')
    axs[i,j].set_xlabel(r'$y = p / T_{\nu,0}$')
    axs[i,j].legend(loc='lower left')
    axs[i,j].set_ylim(1e-5, 1e0)
    # axs[i,j].set_xlim(PRE.LOWER/T_CNB, PRE.UPPER/T_CNB)
    axs[i,j].set_xlim(PRE.LOWER/T_CNB, 1e2)


plt.savefig(f'figures/Fermi_Dirac_{PRE.NUS}neutrinos.pdf')
plt.show()

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(12,12))
fig.suptitle(
    'Integral for number density' 
    '\n'
    f'({PRE.NUS} neutrinos)',
    fontsize=18)

Test_masses = np.array([0.01,0.015,0.02,0.025])*eV

for j, m_eV in enumerate(NU_MASSES):

    k = j

    # Convert to first and last momenta (of each neutrino).
    p0, _ = fct.u_to_p_eV(u_all[:,0], m_eV)
    p1, _ = fct.u_to_p_eV(u_all[:,-1], m_eV)

    # Sort momenta.
    ind = p0.argsort()
    p0_sort, p1_sort = p0[ind], p1[ind]

    curtain_behaviour = False

    if curtain_behaviour:
        p1_final = p1_sort
        p0_final = p0_sort
    else:
        # Each velocity has a batch of neutrinos!
        # Take min. of each to represent most clustered ones.
        p1_blocks = p1_sort.reshape((PRE.Vs, PRE.PHIs*PRE.THETAs))
        p1_final = np.min(p1_blocks, axis=1)
        p0_blocks = p0_sort.reshape((PRE.Vs, PRE.PHIs*PRE.THETAs))
        p0_final = np.min(p0_blocks, axis=1)


    # Fermi-Dirac value with momentum at end of sim.
    FDvals = fct.Fermi_Dirac(p1_final)

    # What number density function integrates.
    y = FDvals * p0_final**2 
    x = p0_final

    i = 0
    if j in (2,3):
        i = 1
        j -= 2


    # Using all velocities of sim.
    axs[i,j].set_title(f'{m_eV} eV')
    axs[i,j].semilogx(x/T_CNB, y, alpha=0.9)

    # Original Fermi-Dirac distr.
    y_FD = fct.Fermi_Dirac(PRE.MOMENTA) * PRE.MOMENTA**2
    x_FD = PRE.MOMENTA/T_CNB
    axs[i,j].semilogx(x_FD, y_FD, c='orange', alpha=0.8, ls='-.')

    # Plot settings.
    axs[i,j].set_xlabel(r'y = $p_0/T_{CNB}$')
    axs[i,j].set_ylabel(r'$FD(p_1) \cdot p_0^2$')
    axs[i,j].set_ylim(0, 6*1e-8)

plt.show()

In [ ]:
def n_nu_of_redshift(m_eV):
    
    # Convert to momenta. 
    p, _ = fct.u_to_p_eV(u_all, m_eV)

    n_nus = np.zeros(len(ZEDS))
    for i, _ in enumerate(ZEDS):

        # Calculate number density.
        n_nus[i] = fct.number_density(p[:,0], p[:,i])

    np.save(f'{out_file}', n_nus)

out_file = f'{PRE.OUT_DIR}/z_dependence_{PRE.NUS}nus_{tot_name}.npy'

# Plotting.
fig, ax = plt.subplots(1,1, figsize=(8,14))

colors = ['blue', 'orange', 'green', 'red']

for j, m in enumerate(NU_MASSES):
    n_nu_of_redshift(m)

    n = np.load(f'{out_file}')/N0
    # print('Max:', np.max(n),'Mean:', np.mean(n))

    ax.semilogy(ZEDS, n-1, c=colors[j])


ax.set_title(f'{PRE.NUS} neutrinos, solver={SOLVER}')
ax.set_xlabel('z')
ax.set_ylabel(r'$n_{\nu} / n_{\nu, 0}$')
ax.set_ylim(1e-3, 3e1)

ax.yaxis.set_major_formatter(ticker.FuncFormatter(fct.y_fmt))
plt.savefig(f'figures/z_dependence_{PRE.NUS}nus_{tot_name}.pdf')
plt.show()